In [ ]:
# goekayyavuz

## import libaries
import requests
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3

try:
    url = 'https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures'
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
except requests.exceptions.RequestException as e:
    print(f"Error while accessing website: {e}")
    exit()

tables = soup.find_all("table")

## find the right table class inside of all tables
for i, table in enumerate(tables):
    print(f"Tabelle {i}: {table.get('class')}")

table = soup.find("table", {"class": "stats_table"})
if not table:
    print("Table not found.")
    exit()


## parse all data inside of variables
if table:
    rows = table.find_all("tr")
    matches = [] # store the variables inside of this list
    for row in rows:
        cols = row.find_all("td")
        if cols:
                dayofweek = cols[0].text.strip()
                date = cols[1].text.strip()
                start_time = cols[2].text.strip()
                home_team = cols[3].text.strip()
                home_xg = cols[4].text.strip()
                score = cols[5].text.strip()
                away_xg = cols[6].text.strip()
                away_team = cols[7].text.strip()
                attendance = cols[8].text.strip()
                venue = cols[9].text.strip()
                referee = cols[10].text.strip()

                matches.append([dayofweek, date, start_time, home_team, home_xg, score, away_xg, away_team, attendance, venue, referee])

columns = ['dayofweek', 'date', 'start_time', 'home_team', 'home_xg', 'score', 'away_xg', 'away_team', 'attendance', 'venue', 'referee']
df = pd.DataFrame(matches,columns=columns) # create dataframe with matches list



## configure sql connection
conn = sqlite3.connect("premier_league.db")
cursor = conn.cursor()  

## sql command to create the table
cursor.execute('''
CREATE TABLE IF NOT EXISTS matches_2425(
               id INTEGER PRIMARY KEY AUTOINCREMENT,
               dayofweek TEXT,
               date TEXT,
               starttime TEXT,
               home_team TEXT,
               home_xg FLOAT,
               score TEXT,
               away_xg FLOAT,
               away_team TEXT,
               attendance INTEGER,
               venue TEXT,
               referee TEXT
                )
                               

''')

## store the dataframe in sql database
df.to_sql("matches_2425", con=conn, if_exists="replace", index=False)
conn.close()

## test if the table is stored correctly
conn = sqlite3.connect("premier_league.db")
cursor = conn.cursor()
query = "SELECT * FROM matches_2425"
pd.read_sql(query,conn)




In [2]:
df

,dayofweek,date,start_time,home_team,home_xg,score,away_xg,away_team,attendance,venue,referee
0,Fri,2024-08-16,20:00,Manchester Utd,2.4,1–0,0.4,Fulham,"73,297",Old Trafford,Robert Jones
1,Sat,2024-08-17,12:30,Ipswich Town,0.5,0–2,2.6,Liverpool,"30,014",Portman Road Stadium,Tim Robinson
2,Sat,2024-08-17,15:00,Newcastle Utd,0.3,1–0,1.8,Southampton,"52,196",St James' Park,Craig Pawson
3,Sat,2024-08-17,15:00,Nott'ham Forest,1.3,1–1,1.2,Bournemouth,"29,763",The City Ground,Michael Oliver
4,Sat,2024-08-17,15:00,Everton,0.5,0–3,1.4,Brighton,"39,217",Goodison Park,Simon Hooper
...,...,...,...,...,...,...,...,...,...,...,...
415,Sun,2025-05-25,16:00,Fulham,,,,Manchester City,,Craven Cottage,
416,Sun,2025-05-25,16:00,Nott'ham Forest,,,,Chelsea,,The City Ground,
417,Sun,2025-05-25,16:00,Manchester Utd,,,,Aston Villa,,Old Trafford,
418,Sun,2025-05-25,16:00,Wolves,,,,Brentford,,Molineux Stadium,


In [6]:
columns_team = ['TeamName','GamesPlayed', 'Win', 'Draw', 'Loss', 'GoalScored', 'GoalAgainst', 'GoalDifference', 'Points']
#df_points_temp = pd.DataFrame(matches,columns=columns_team)

df_points_temp = pd.DataFrame()
df_points_temp['TeamName'] = df.home_team
df_points_temp['Win'] =

print(df_points_temp)



            TeamName
0     Manchester Utd
1       Ipswich Town
2      Newcastle Utd
3    Nott'ham Forest
4            Everton
..               ...
415           Fulham
416  Nott'ham Forest
417   Manchester Utd
418           Wolves
419        Tottenham

[420 rows x 1 columns]
